In [1]:
import sys, os
sys.path.insert(0, os.path.abspath("../.."))
%load_ext elastic_notebook

In [2]:
%%RecordEvent
import numpy as np
from sklearn import datasets
from sklearn.neighbors import KNeighborsClassifier

<class 'IPython.core.interactiveshell.ExecutionResult'>


In [3]:
%%RecordEvent
iris_X, iris_y = datasets.load_iris(return_X_y=True)

<class 'IPython.core.interactiveshell.ExecutionResult'>


In [4]:
%%RecordEvent
np.random.seed(0)

<class 'IPython.core.interactiveshell.ExecutionResult'>


In [5]:
%%RecordEvent
indices = np.random.permutation(len(iris_X))
iris_X_train = iris_X[indices[:-10]]
iris_y_train = iris_y[indices[:-10]]
iris_X_test = iris_X[indices[-10:]]
iris_y_test = iris_y[indices[-10:]]

<class 'IPython.core.interactiveshell.ExecutionResult'>


In [6]:
%%RecordEvent
indices = 123
knn = KNeighborsClassifier()
knn.fit(iris_X_train, iris_y_train)

knn.predict(iris_X_test)

array([1, 2, 1, 0, 0, 0, 2, 1, 2, 0])

<class 'IPython.core.interactiveshell.ExecutionResult'>


In [7]:
%%RecordEvent
def im_a_function(x):
    print(x + 1)
im_a_function(indices)

124
<class 'IPython.core.interactiveshell.ExecutionResult'>


In [8]:
%Inspect

---------------------------
VSs:
np 0
datasets 0
KNeighborsClassifier 0
iris_X 0
iris_y 0
indices 0
indices 1
iris_X_train 0
iris_y_train 0
iris_X_test 0
iris_y_test 0
knn 0
im_a_function 0
---------------------------
OEs:
---------------------------
OE num: 0  runtime: 0.3076648712158203
sources
destinations
    np 0
    datasets 0
    KNeighborsClassifier 0
---------------------------
OE num: 1  runtime: 0.0023038387298583984
sources
    datasets 0
destinations
    iris_X 0
    iris_y 0
---------------------------
OE num: 2  runtime: 0.00014901161193847656
sources
    np 0
destinations
---------------------------
OE num: 3  runtime: 0.0006048679351806641
sources
    iris_y 0
    iris_X 0
    np 0
destinations
    indices 0
    iris_X_train 0
    iris_y_train 0
    iris_X_test 0
    iris_y_test 0
---------------------------
OE num: 4  runtime: 0.004956960678100586
sources
    iris_X_test 0
    iris_y_train 0
    KNeighborsClassifier 0
    iris_X_train 0
destinations
    indices 1
    

In [9]:
%Checkpoint checkpoints/numpy.pickle

minimum migration cost: 0.020161777420043946
---------------------------
variables to migrate:
KNeighborsClassifier 1472
knn 48
np 72
datasets 72
indices 28
---------------------------
variables to recompute:
iris_X_test 448
iris_X_train 4608
iris_y 1312
iris_y_test 192
im_a_function 136
iris_y_train 1232
iris_X 4928
---------------------------
OEs to recompute:
5 0.0003330707550048828
1 0.0023038387298583984
3 0.0006048679351806641
Checkpoint saved to: checkpoints/numpy.pickle
